<a href="https://colab.research.google.com/github/raminicano/SMU_GM/blob/yeowon/link_prediction_gnn_epoch150.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#파이썬 버전 체크
import sys
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [2]:
import torch

In [3]:
!pip3 install torch torchvision

In [4]:

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:2.0.1+cu118
cuda version: 11.8
cudnn version:8700


In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# Link Prediction using Graph Neural Networks
Predicts whether an edge exists between two particular nodes. e.g. social recommendation, item recommendation

The link prediction task here is formulated as a binary classification problem:
- Treat existing edges in the graph as *positive* examples
- Treat non-existing edges in the graph as *negative* examples
- Divide the *positive* and *negative* examples into  a training and test set
- Evaluate the model with Area Under Curve (AUC)

In [6]:
# dgl 설치 안될 때
!pip install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html


Looking in links: https://data.dgl.ai/wheels/cu118/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 MB 9.8 MB/s eta 0:00:00


In [7]:
torch.version.cuda

'11.8'

In [8]:
%%capture
!pip3 install dgl-cu118

In [9]:
import json
import h5py
import time
import itertools

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import scipy.sparse as sp
import networkx as nx
import dgl
import dgl.function as fn
from dgl.nn import SAGEConv
from sklearn.metrics import roc_auc_score


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.9/884.9 kB 70.2 MB/s eta 0:00:00


In [12]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910460 sha256=3edbf0c18e6397e90dab08f9bdac9dd317440b1363e55004874fa442add0d34f
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [13]:

import tensorflow as tf
from keras.layers import Layer, Dense
from torch_geometric.nn.inits import glorot, zeros

# 0. GPU Connection

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device("cpu"))
print('device : ', device)

device :  cuda:0


# 1. Data Preparation

In [15]:
abs_path = './drive/MyDrive/Colab Notebooks'

In [16]:
seq = []
emb = []
emb_mean = []
with h5py.File(abs_path + "/SMWU/per-protein.h5", "r") as file:
    print(f"number of entries: {len(file.items())}")
    for sequence_id, embedding in file.items():
        seq.append(sequence_id)
        emb.append(embedding)
        emb_mean.append(np.array(embedding).mean())
        print(
            f"  id: {sequence_id}, "
            f"  embeddings shape: {embedding.shape}, "
            f"  embeddings mean: {np.array(embedding).mean()}"
        )

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  id: Q96KR4,   embeddings shape: (1024,),   embeddings mean: 0.0012369155883789062
  id: Q96KR6,   embeddings shape: (1024,),   embeddings mean: -0.0012950897216796875
  id: Q96KR7,   embeddings shape: (1024,),   embeddings mean: 0.00028896331787109375
  id: Q96KS0,   embeddings shape: (1024,),   embeddings mean: 9.137392044067383e-05
  id: Q96KS9,   embeddings shape: (1024,),   embeddings mean: 0.00019478797912597656
  id: Q96KT0,   embeddings shape: (1024,),   embeddings mean: -0.0005655288696289062
  id: Q96KT6,   embeddings shape: (1024,),   embeddings mean: 0.0003209114074707031
  id: Q96KT7,   embeddings shape: (1024,),   embeddings mean: -0.00017654895782470703
  id: Q96KV6,   embeddings shape: (1024,),   embeddings mean: 0.001007080078125
  id: Q96KV7,   embeddings shape: (1024,),   embeddings mean: 0.0018854141235351562
  id: Q96KW2,   embeddings shape: (1024,),   embeddings mean: 0.00262451171875
  id: Q96KW9,   embeddings shape: (1024,), 

In [17]:
f = open(abs_path + '/SMWU/homo_gene2acc_json.json')
gene_to_id = json.load(f)

nf_dict= {}
for i, gene in enumerate(gene_to_id):
    gid = gene_to_id[gene]
    for j, s in enumerate(seq):
        if gid != '-':
            if s == gid:
                nf_dict[gene] = emb_mean[j]

In [18]:
ppi = pd.read_csv(abs_path+'/SMWU/context-PPI_final.csv')

all_nodes = list(set(list(ppi['gene_a'].unique()) + list(ppi['gene_b'].unique())))

In [19]:
def dropout(X, drop_prob):
    assert 0 <= drop_prob <= 1
    # In this case, all elements are dropped out
    if drop_prob == 1:
        return X.zeros_like()
    mask = nd.random.uniform(0, 1, X.shape) > drop_prob
    return mask * X / (1.0-drop_prob)

# 2. Define GNN Model
GraphSAGE in this case

In [20]:
# two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'lstm')
        self.conv2 = SAGEConv(h_feats, h_feats, 'lstm')

    def forward(self, g, in_feats):
        h = self.conv1(g, in_feats)
        h = F.tanh(h)
        h = self.conv2(g, h)
        return h

In [21]:
from __future__ import print_function
from __future__ import division

Use `DotPredictor` or `MLP` to compute new edge features based on the original node/edge features

In [22]:
class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

Create empty graph and add nodes

In [23]:
g = dgl.DGLGraph() #그래프데이터 다루는 최적화 라이브러리 DGL
g.add_nodes(len(all_nodes))

/usr/local/lib/python3.10/dist-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


Give each unique gene an index so as to define edges from Gene A to Gene B

In [24]:
id_dict = {}
for i, gene in enumerate(all_nodes):
    id_dict[gene] = i

#genea_idx = []

#for i, gene in enumerate(ppi['gene_a']):
#    for j, idx_match in enumerate(id_dict):
#        if gene == idx_match:
#            genea_idx.append(id_dict[idx_match])
#geneb_idx = []
#for i, gene in enumerate(ppi['gene_b']):
#    for j, idx_match in enumerate(id_dict):
#        if gene == idx_match:
#            geneb_idx.append(id_dict[idx_match])


In [25]:
genea_idx = torch.tensor([id_dict[val] for val in ppi['gene_a']]).long()
geneb_idx = torch.tensor([id_dict[val] for val in ppi['gene_b']]).long()



In [26]:
g.add_edges(torch.tensor(genea_idx).long(), torch.tensor(geneb_idx).long())

<ipython-input-26-afefe84080ee>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  g.add_edges(torch.tensor(genea_idx).long(), torch.tensor(geneb_idx).long())


In [27]:
#nx.draw(g.to_networkx(), with_labels = True)

Create Edge and Node Features <br>
Initialize node features as zeros and use edge features to update them

In [28]:
print("We have %d nodes. " % g.number_of_nodes())
print("We have %d edged. " % g.number_of_edges())

We have 15693 nodes. 
We have 229934 edged. 


In [29]:
ppi

,id,gene_a,gene_b,pid,cell_name,cell_category,cell_sex,cell_species
0,0,ALDH1A1,ALDH1A1,25416956,HEK,Cancer cell line,Female,Homo sapiens
1,7,PIK3R2,ERBB2,16729043,Hs 912.T,Cancer cell line,Female,Homo sapiens
2,8,PTPN18,ERBB2,25081058,Hep-G2,Cancer cell line,Male,Homo sapiens
3,10,SMURF2,ARHGAP5,28514442,MCF-10A,Spontaneously immortalized cell line,Female,Homo sapiens
4,11,NF2,ERBB2,28628118,NCI-H1299,Cancer cell line,Male,Homo sapiens
...,...,...,...,...,...,...,...,...
229929,391402,STK38L,DPCD,30108113,T-REx-293,Transformed cell line,Female,Homo sapiens
229930,391404,MAPK6,BOLA2,26972000,HEK293T,Transformed cell line,Female,Homo sapiens
229931,391405,HSPD1,MRPL41,29568061,T-REx-293,Transformed cell line,Female,Homo sapiens
229932,391407,MKI67,MTHFD1L,26949251,BJ1-hTERT,Telomerase immortalized cell line,Male,Homo sapiens


In [30]:
cell_cat = torch.tensor(pd.get_dummies(ppi['cell_category']).values)
cell_sex = torch.tensor(pd.get_dummies(ppi['cell_sex']).values)
cell_spe = torch.tensor(pd.get_dummies(ppi['cell_species']).values)

In [31]:
cell_cat

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1]], dtype=torch.uint8)

In [32]:
cell_spe.shape

torch.Size([229934, 1])

In [33]:
ef = torch.cat([cell_cat, cell_sex, cell_spe], dim=-1).float()

In [34]:
feat = []
id_feat = []
ids = []
for i, gene_id in enumerate(id_dict.keys()):
    for j, gene_nf in enumerate(nf_dict.keys()):
        if gene_nf == gene_id:
            feat.append([nf_dict[gene_nf]])
            id_feat.append([id_dict[gene_id], nf_dict[gene_nf]])
            ids.append(id_dict[gene_id])
#node_remove = []
#for i in range(len(all_nodes)):
#    if i not in ids:
#        node_remove.append(i)

In [35]:
node_remove = [num for num in range(len(all_nodes)) if num not in ids]

len(node_remove)

687

In [36]:
no_nf = True
if no_nf:
    g.edata['feat'] = ef
    g.ndata['_feat'] = torch.zeros(g.num_nodes(), ef.size(1))
    node_dim = g.ndata['_feat'].size(1)
    edge_dim = g.edata['feat'].size(1)
    latent_dim = 5
    node_encoder = nn.Linear(node_dim, latent_dim)
    edge_encoder = nn.Linear(edge_dim, latent_dim)
    g.ndata['_h'] = node_encoder(g.ndata['_feat'])
    g.edata['_h'] = edge_encoder(g.edata['feat'])
    g.pull(g.nodes(),
        message_func=fn.copy_e('feat', 'm'),
        reduce_func=fn.sum('m', 'feat'))
else:
    g = dgl.remove_nodes(g, node_remove)
    g.ndata['feat'] = torch.tensor(feat).float()



In [37]:
# Split edge set for training and testing
u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())), shape=(g.number_of_nodes(), g.number_of_nodes()))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]


In [38]:
train_g = dgl.remove_edges(g, eids[:test_size])

In [39]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [40]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16).to(device)

# 3. Training Methods

In [41]:
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16).to(device)
pred = MLPPredictor(16).to(device)

In [42]:
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

In [43]:
def compute_loss(pos_score, neg_score, device):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).to(device)
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score): #유효성검사
    scores = torch.cat([pos_score, neg_score]).cpu().numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

# 4. Model Training

In [ ]:
# Initialize lists to store training and test losses
train_losses = []
test_losses = []
train_aucs = []
test_aucs = []
epoch = 70

# ----------- training -------------------------------- #
all_logits = []
for e in range(70):
    # forward
    train_g = train_g.to(device)
    train_pos_g = train_pos_g.to(device)
    train_neg_g = train_neg_g.to(device)

    h = model(train_g, train_g.ndata['feat']).to(device)
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score, device)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

    # Store the training loss and AUC at each epoch
    train_losses.append(loss.item())
    train_auc = compute_auc(pos_score.detach(), neg_score.detach())
    train_aucs.append(train_auc)

    # ----------- check results ------------------------ #
    with torch.no_grad():
      test_pos_g = test_pos_g.to(device)
      test_neg_g = test_neg_g.to(device)

      pos_score = pred(test_pos_g, h)
      neg_score = pred(test_neg_g, h)

      test_loss = compute_loss(pos_score, neg_score, device)
      test_losses.append(test_loss.item())

      test_auc = compute_auc(pos_score.detach(), neg_score.detach())
      test_aucs.append(test_auc)

      print('AUC', test_auc)




In epoch 0, loss: 0.6825987100601196
AUC 0.7514615582233111
AUC 0.7553805312238338
AUC 0.7695184314338726
AUC 0.7977352867170016
AUC 0.8114015681403863
In epoch 5, loss: 0.5723692178726196
AUC 0.8180479240968069
AUC 0.8217941142474028
AUC 0.8260220219583961
AUC 0.8308534292862233
AUC 0.835358178451627
In epoch 10, loss: 0.5087370276451111
AUC 0.8403082373862661
AUC 0.8435545571516626
AUC 0.8459154013409775
AUC 0.8474432924677755
AUC 0.8480546267204676
In epoch 15, loss: 0.4893193542957306
AUC 0.8490325678341148


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
h = model(train_g, train_g.ndata['feat']).to(device)
h

In [ ]:
# Plot the training and test losses
epochs = range(0, epoch)  # x-axis values (epochs)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.show()

# Plot the training and test AUC curves
plt.plot(epochs, train_aucs, label='Training AUC')
plt.plot(epochs, test_aucs, label='Test AUC')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.title('Training and Test AUC')
plt.legend()
plt.show()
